In [10]:
import pickle
from sklearn import datasets

x =  datasets.make_friedman1(n_samples = 50, n_features = 5, noise = 0)
pickle.dump(x, '~/friedman1')

TypeError: file must have a 'write' attribute

In [ ]:
# import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
import time

#use simulated regression dataset to build random forest and use built in algorithms to find the
#best parameters 
x =  make_friedman1(n_samples = 50, n_features = 4, noise = 0)
par_grid = {'n_estimators':np.linspace(1,5,5), 
        'min_samples_leaf':np.linspace(0.1,0.5,5), 
        'max_features':np.linspace(0.1,0.5,), 
        'max_samples':[0.01,0.1,0.2,0.3]}

rf = RandomForestRegressor()
gs = model_selection.GridSearchCV(estimator = rf, param_grid = par_grid, cv = 3)
%time gs.fit(x[0], x[1])
print(gs.best_params_)
print(gs.best_score_)

rs = model_selection.RandomizedSearchCV(estimator = rf, 
                                    param_distributions = par_grid
                                    n_iter = 100, 
                                    cv = 3)
%time rs.fit(x[0], x[1])
print(rs.best_params_)
print(rs.best_score_)